In [96]:
from daily_processes import generate_dictionary

notes_df,examine_further= generate_dictionary()

In [97]:
notes_df

,Process,Categorization,Word,Definition,Source
0,ML Definitions,Model Type,Types of ML,"Within the machine learning paradigm, models a...",Notes DF
148,ML Definitions,General Principles,XOR,Logical operation that outputs true when exact...,Examine Further
11,ML Model,Definition,Definition,Self Explanatory,Insert Records
12,ML Model,Definition,Definition,Decision Strategy: A Decision Strategy is a me...,Insert Records2
13,ML Model,Definition,Definition,Anchoring Effect: A cognitive bias where peopl...,Insert Records2
...,...,...,...,...,...
399,Model Type,Parameter,Neural Models,Models which learn representations through lay...,Examine Further
400,Model Type,Parameter,Nonlinear Models,Models which capture relationships that cannot...,Examine Further
401,Model Type,Parameter,Probabilistic Models,Models which explicitly represent uncertainty ...,Examine Further
402,Model Type,Parameter,Rule-base Models,Models which operate using explicit if–then lo...,Examine Further


In [ ]:
# IS MY DAILY AUTOMATION PROGRAM WORKING. 
# IF Yes, Why Does REVIEW, TEST and DAILY NOTES NOT Update.

In [ ]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

In [94]:
links['google_definition_csv']

'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

In [93]:
from data_d_dicts import links
df = pd.read_csv(links['google_definition_csv'])
df.head()

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Classification,Model Type
0,Data Preparation,Definition,Baseline,Baseline represents the typical or expected be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Preparation,Definition,Dimension,A Dimension is a conceptual axis of behavior o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data Preparation,Definition,Implementation,An Implementation is a specific measurable rep...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Preparation,Definition,Momentum,Momentum represents the direction and rate of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Preparation,Definition,Parameter,A Parameter is a tuning choice that controls h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df[

In [88]:
df[df['Word']=='Evaluation']['Definition'].item()

'Evaluation measures how well the final model performs against defined success metrics on a truly unseen test dataset. Metrics depend on the problem type (e.g., accuracy, AUC, RMSE, precision-recall). This step provides an objective assessment of whether the model meets business or research requirements. Evaluation results often determine whether a model is production-ready.'

In [87]:
df[df['Word']=='Learning Type']['Definition'].item()

'Describes how the model or technique learns from data, specifically the nature of the labels or feedback it receives during training. It distinguishes whether learning is driven by known outcomes (Supervised), structure discovery without labels (Unsupervised), or performance assessment rather than learning itself (Evaluation). This column anchors where in the ML lifecycle a concept operates.'

In [60]:
df.head()

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Classification,Model Type
0,Data Preparation,Definition,Baseline,Baseline represents the typical or expected be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Preparation,Definition,Dimension,A Dimension is a conceptual axis of behavior o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data Preparation,Definition,Implementation,An Implementation is a specific measurable rep...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Preparation,Definition,Momentum,Momentum represents the direction and rate of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Preparation,Definition,Parameter,A Parameter is a tuning choice that controls h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
def generate_df_documentation(df,
                              data_dictionary=None,
                              column_name="",
                              match_on="Word",
                              text_description='Definition',):
    
    '''

    

    '''

    # Make sure there is a Dictionary for Information.
    try:
        dd_df = data_dictionary.copy()
    except:
        dd_df = pd.read_csv(links['google_definition_csv'])
        dd_df = dd_df[[match_on,text_description]].copy()

    if len(column_name) == 0:
        # Making a Dictionary of the Dataframe Column Values
        final_df = pd.DataFrame(df.columns.tolist(),columns=[match_on])
        final_df = final_df.merge(dd_df,on=match_on,how='left')
        
    else:
        # Making a dictionary of the Column Name 

        col_values = sorted(
            df[column_name]
            .dropna()
            .astype(str)
            .str.strip()
            .loc[lambda s: s != ""]
            .unique())
        
        final_df = pd.DataFrame(col_values,columns=[match_on])

    return final_df.merge(dd_df[[text_description,match_on]],on=match_on,how='left')

generate_df_documentation(df,
                          data_dictionary=df,
                          column_name='Learning Type')

,Word,Definition
0,Evaluation,Evaluation measures how well the final model p...
1,Supervised Learning,Supervised learning trains a model using label...
2,Unsupervised Learning,Unsupervised learning works with unlabeled dat...


In [92]:
df[df['Process']=='Data Preparation']

,Process,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Classification,Model Type
0,Data Preparation,Definition,Baseline,Baseline represents the typical or expected be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Preparation,Definition,Dimension,A Dimension is a conceptual axis of behavior o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Data Preparation,Definition,Implementation,An Implementation is a specific measurable rep...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Preparation,Definition,Momentum,Momentum represents the direction and rate of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Preparation,Definition,Parameter,A Parameter is a tuning choice that controls h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Data Preparation,Definition,Volatility,Volatility represents the consistency or varia...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Data Preparation,Regularization,ElasticNet,Approach which combines L1 and L2 Regularization.,NaN,NaN,NaN,$$\nJ(\theta) = \mathrm{MSE}(\theta)\n+ r \lef...,"Small, Medium",NaN,NaN,NaN
7,Data Preparation,Regularization,Lasso,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,$$\nJ(\theta) = \sum_{i=1}^{m} \left( y_i - \h...,"Small, Medium",NaN,NaN,NaN
8,Data Preparation,Regularization,Ridge,Technique used to prevent overfitting by addin...,The equation that you use to calculate the cos...,NaN,NaN,$$\nJ(\theta) = \sum_{i=1}^{m} \left( y_i - \h...,"Medium, Large",NaN,NaN,NaN
294,Data Preparation,Semantic Type,State,State represents the current condition of an e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
['Learning Type','Algorithm Classification','Model Type']

In [41]:
def iterate_list_to_str(list_,
                        start_text='',
                        additional_text=', ',
                        ending_text=""):
    text = start_text
    
    for count,word in enumerate(list_):
        if text=="":
            text += word
        else:
            text += additional_text + word 
    
    return text

iterate_list_to_str(['Learning Type','Algorithm Classification', "Model Type"])

'Learning Type, Algorithm Classification, Model Type'

In [49]:
list_ = ['Linear','Nonlinear','Tree-Based','Ensemble','Instance-Based','Probabilistic','Graph-Based','Neural','Kernel-Based',
         'Rule-Based','Meta-Model']

for word in list_:
    print(df[df['Word'].str.contains(word.lower(),case=False)])

         Process Categorization                            Word  \
19   Mathematics        Concept                       Linearity   
23   Mathematics       Property                     Colinearity   
24   Mathematics        Theorem  Best Linear Unbiased Estimator   
117     ML Model      Algorithm               Linear Regression   
118     ML Model      Algorithm      LinearDiscriminantAnalysis   
119     ML Model      Algorithm                       LinearSVC   
120     ML Model      Algorithm                       LinearSVR   
121     ML Model      Algorithm          LocallyLinearEmbedding   
392  Mathematics     Definition               Multicollinearity   

                                            Definition  \
19   Relationship between Independent and Dependent...   
23    No perfect linear relationship between residuals   
24   The Best Linear Unbiased Estimator (BLUE) is a...   
117  Models the relationship between an independent...   
118                                    

In [48]:
for word in ['Learning Type','Algorithm Classification', "Model Type"]:
    print(f"Column Classification, {word}: Current List Options:{df[word].unique()} ")

Column Classification, Learning Type: Current List Options:[nan 'Supervised' 'Unsupervised' 'Evaluation'] 
Column Classification, Algorithm Classification: Current List Options:[nan 'Classification' 'Clustering' 'Regression' 'Dimensionality reduction'
 'Dimension Reduction' 'Feature Transformation'
 'Classification or Regression' 'Decision Strategy'] 
Column Classification, Model Type: Current List Options:[nan 'Linear' 'Denisty Based Clustering' 'Parametric Regression'
 'Instance Based Learning' 'Non Linear' 'Tree' 'Tree-Based'
 'Feature Selection' 'Ensemble' 'Meta-Model' 'Probabilistic clustering'
 'Graph-based clustering'] 


In [30]:
df['Learning Type'].unique()

array([nan, 'Supervised', 'Unsupervised', 'Evaluation'], dtype=object)

1. Do I have a Good Search for Format Type Function?
    1. DFColumnTypeDict technically is, but not great.
    2. Review Recently implemented at work for _____ (I can remember 

In [22]:
word_list = []

gpt_question(word_list)

For the following words, can you please provide me for each word a definition which is 4-5 sentences highlighting the definition,
its origin, its importance and application. If the word is a ML Model, can you also please define the following in point form,
type of ML learning, type of ML problem, size of data set it works best on, and anything else which might be appropriate.
Unique Words: heteroskedastic, non-Gaussian, multicollinearity, BisectingKMeans, LocallyLinearEmbedding, NeighborhoodComponentsAnalysis
    


In [2]:
from daily_processes import review_test_results,daily_test

#review_test_results()
daily_test()

Word 1
Category: ML Model
Classification: Algorithm
Word: StackingClassifier



Update Google Sheet with Definition
 


Word 2
Category: ML Definitions
Classification: General Principles
Word: XOR



######################### ANSWER QUESTION ######################### f


Definition: Not Linearly Seperable. Problem for ML. Nand as Function of (x1,x2) new function (h1, h2) - 
Notes: 
Link: 
Markdown: 
Dataset Size: 
Learning Type: 
Alogrithm Class: 



...

Definition: Structural represents relatively stable attributes that describe how an entity is configured within the system rather than how it behaves at a given moment. It answers the question “What is the entity’s structural position?” by reflecting enduring characteristics such as product holdings, account composition, or long-term balances. Structural measures anchor and constrain behavior by defining capacity, access, and configuration. In modeling, Structural variables provide important context but should be included sparingly to avoid overwhelming activity-based signals or unintentionally clustering entities by size rather than behavior. Product Count, Product Flags, Tenure.
Notes: 
Link: 
Markdown: 
Dataset Size: 
Learning Type: 
Alogrithm Class: 



What was the result? (P/F)
 f


Word 5
Category: ML Model
Classification: Algorithm
Word: PLSCanonical



Update Google Sheet with Definition
 f


,Date,Temp_Score,Historical_Score,Word,Process,Categorization,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,2026-01-07,0,-3,SGDClassifier,ML Model,Algorithm,SGDClassifier is a linear classification algor...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
1,2026-01-07,0,-3,VotingClassifier,ML Model,Algorithm,VotingClassifier is an ensemble method that co...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Classification,Meta-Model,1.0
2,2026-01-07,0,-3,Quantile Random Forest,ML Model,Algorithm,Quantile Random Forest is an extension of rand...,NaN,NaN,NaN,NaN,NaN,Supervised,Regression,Tree,NaN
3,2026-01-07,0,-3,RandomTreesEmbedding,ML Model,Algorithm,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
4,2026-01-07,0,-3,Area Under the Curve,Model Evaluation,Feature Selection,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,0,-3,Perceptron,ML Model,Algorithm,The Perceptron is one of the earliest linear c...,NaN,NaN,NaN,NaN,Large,Supervised,Classification,Linear,NaN
6,2026-01-08,0,-3,RidgeCV,ML Model,Algorithm,RidgeCV is a regression model that combines ri...,NaN,NaN,NaN,NaN,"Medium, Large",Supervised,Regression,Linear,NaN
7,2026-01-08,0,-3,Regularization,ML Model,Definition,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,0,-3,SelectFpr,ML Model,Algorithm,SelectFpr is a feature selection method that s...,NaN,NaN,NaN,NaN,Large,NaN,NaN,NaN,NaN
9,2026-01-08,0,-3,AdaBoostClassifier,ML Model,Algorithm,AdaBoostClassifier is an ensemble boosting alg...,NaN,NaN,NaN,NaN,"Small, Medium",Supervised,Classification,Linear,1.0


In [ ]:
Move Batch and API

In [3]:


from data_d_strings import gpt_question
for word in words:
    print(gpt_question(word))
    



For the definition StackingClassifier can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition XOR can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition Batch Processing can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition API Processing can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition KNeighborsClassifier can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition PLSCanonical can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.
For the definition RandomTreesEmbedding can you please provide me a 4-5 sentences h

In [ ]:
######################################

# Action Add DQ Item.
# Add Read Me, which Explanation of process and Definitions of all Categorizations.

# DQ Validation Component

# Question I need to ask Myself.
# Is it a Definition.
# Is it a Step in a Process
# Is it a Process
# Is it information Related to a Process.

# DQ Process 1: Need to Avoid Duplication of Word in ALL SHEETS.
# Definitions: Are Technical Definitions, descriptions and eloborate Notes describing a Term. 
# Notes: Merge in Definitions.
# Take Longer Listings and organize them, Models, BLUE, 

######################################

In [ ]:
def generate_read_me():

    from data_d_dicts import links
    df = pd.read_csv(links['google_definition_csv'])

    read_me_df = pd.DataFrame(df.columns,columns=['W'])

    return read_me_df
    

In [ ]:
read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df.merge(df[['Category','Categorization','Definition','Word']],on='Word',how='left')

read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df
